# CASO DE ESTUDIO

La empresa x quiere implementar una herramienta que permita recomendar a sus usuarios un inmueble a partir de la información que este seleccione desde un formulario con preguntas ya establecidas por la entidad. Basado en las respuestas que escogió el usuario, la herramienta debe mostrar un listado con por lo menos 5 recomendaciones que se ajusten al usuario en orden de mayor a menor probabilidad de acierto para el gusto del usuario. En todos los casos se debe mostrar mínimo 5 recomendaciones así la posibilidad de que el usuario este a gusto con la recomendación sea cercana a cero.

Es importante resaltar que la empresa apenas inicia en el negocio, por esta razón no se cuenta con información alguna de casos de elección de los usuarios para tomar como base para análisis y por consiguiente no se tiene feedback alguno. Los únicos datos con que se cuentan son los relacionados con los inmuebles que es el producto final que se quiere comerciar.

## ANÁLISIS Y SOLUCIÓN

__Analice el problema y defina qué campo de la inteligencia artificial usaría para abordar el tema y el porqué de su elección.__

_Teniendo en cuenta que no se tiene feedback alguno del proceso de elección de los usuarios, es necesario considerar la aplicación de algoritmos de tipo __no-supervisado__, ya que estos son capaces de encontrar patrones dentro de la data cuando no se cuenta con una variable de tipo clase._

__Cuál o cuáles métodos se ajustan más a la solución que propone en el campo de la inteligencia artificial elegido y explique el porqué de su elección.__

_De acuerdo a lo anterior, la metodología seleccionada para aplicar sobre la data es la de __clustering__, cuyo objetivo es el de construir grupos (clusters) de datos que siguen un mismo patrón. Esto quiere decir, que para el caso de estudio, el algoritmo se encarga de crear grupos de inmuebles similares._

_El algoritmo para la creación de clusters será el __K-means__, que consiste en división de las observaciones (los inmuebles) en un número determinado de clusters, en donde cada observación pertenece al cluster al que más se aproxime según la distancia euclediana._

__La encuesta debe contener mínino 7 preguntas y máximo 12 preguntas (las preguntas son indiferentes, por esto no es necesario crear preguntas, pero si lo cree necesario escríbalas para dar sustento a su análisis). Sugiera cuantas características debería tener cada pregunta de la encuesta, se deben usar variables de tipo numéricas, tipo nominal y tipo SI/NO.__

_Teniendo que en la actualidad existen diferentes herramientas que permiten filtrar los inmuebles disponibles basados en los aspectos técnicos de preferencia del usuario, se decidió que con el objetivo de lograr una mayor compatibilidad entre clientes e inmuebles se deben tener en cuenta tanto los aspectos técnicos como los sentimentales (las prioridades)._

_Por lo anterior, se sugiere que la encuesta tenga dos secciones distintas, así:_
- ___Sección técnica:___ _Dentro de esta sección se encuentran entre 6 y 7 preguntas, cuyo objetivo es conocer los requerimientos físicos y explícitos de cada cliente, como lo es el número de habitaciones o el tipo de vivienda. Teniendo en cuenta que la creación de los clusters se hace usando valores numericos, se sugiere que las variables sean de tipo numerico siempre que sea posible. No obstante, y con el fin de no confundir a los clientes, algunas variables pueden ser de tipo nominal, como lo es por ejemplo el tipo de inmueble ("casa" o "apartamento"). En estos casos, se realiza un trabajo de pre procesamento donde se convierten las variables nominales, en numéricas (ej: "casa" = 1, "apartamento" = 0)._<br><br>
- ___Sección de prioridades:___ _En esta sección el objetivo es conocer cuáles variables, a la hora de escoger un inmueble, son más importantes para el cliente. En el presente estudio se tienen en cuenta 5 variables (años de construido, amplitud, precio, ubicación y zonas verdes), las cuales el cliente debe organizar de 1 a 5, siendo 1 la que más tiene en cuenta a la hora de seleccionar un inmueble._

__Basado en el punto anterior proponga un modelo que permita solucionar el caso de estudio y explique cuál es la importancia de cada ítem del modelo, cuanto aporta porcentualmente cada ítem al modelo y cuál fue la razón de asignarle esos pesos a cada uno.__

_Para la elaboración del presente modelo se extrajeron los datos de 5710 inmuebles de la página www.metrocuadrado.com. Los datos disponibles de cada inmueble, los cuales fueron convertidos a variables numéricas cuando era necesario, se encuentran a continuación:_

- ___Tipo de inmueble:___ _Casa (1), Apartamento (0)._
- ___Años de construido:___ _En construcción (5), Para estrenar (4), Entre 0 y 5 años (3), Entre 5 y 10 años (2), Entre 10 y 20 años (1), Más de 20 años (0)._
- ___Área:___ _Se dejó el valor indicado en cada inmueble. Se eliminó el ruido._
- ___Habitaciones___
- ___Baños___
- ___Parqueaderos___
- ___Estrato___
- ___Zonas verdes:___ _Sí (1), No (0)._
- ___Precio___

_Se utilizaron las primeras 6 variables (tipo, años, área, habitaciones, baños y parqueaderos) para entrear el modelo y crear los clusters, ya que son éstas variables las que determinar los requisitos explícitos de los clientes._

_Por otro lado, para la selección de los inmuebles a sugerir, se utilizaron las prioridades de los clientes, mediante el cálculo de pesos, usando la siguiente fórmula:_

> __prioridad1 x 0.8 + prioridad2 x 0.6 + prioridad3 x 0.4 + prioridad4 x 0.2__

_Donde la prioridad1 corresponde a la variable que el cliente posicionó en primer lugar, la prioridad2 es la que colocó en el siguiente lugar, y así sucesivamente._

__Implemente la solución en Python al problema propuesto indicando la versión de Python usada para el ejercicio. __

In [329]:
#Python 3.0

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

In [330]:
#Read data
data_orig = pd.read_csv('metrocuadrado.csv')
data = pd.read_csv('metrocuadrado_clusters.csv')
X_clusters = data.iloc[:,:6]
X_mean = X_clusters.mean()
X_std = X_clusters.std()
X_clusters = (X_clusters-X_mean)/ X_std
X_clusters = X_clusters.fillna(0)

In [313]:
#Train model
est = KMeans(n_clusters=20)

clusters = est.fit(X_clusters)
labels = clusters.labels_

In [314]:
#Add labels to data
data['clusters'] = labels

In [315]:
def predict(input_y,priorities,data,data_orig):
    
    #Select cluster
    Y = est.predict(input_y)

    #Normalize values from property list
    pl = data.loc[data.clusters == Y[0]]
    pl["area"] = (pl["area"]-pl["area"].min())/pl["area"].max()
    pl["anos_construido"] = (pl["anos_construido"]-pl["anos_construido"].min())/pl["anos_construido"].max()
    pl["estrato"] = (pl["estrato"]-pl["estrato"].min())/pl["estrato"].max()
    pl["precio"] = ((1/pl.loc[:,"precio"])-(1/pl.loc[:,"precio"]).min())/(1/pl.loc[:,"precio"]).max()

    #Select 5 properties according to weights
    indexes = pl.loc[:,priorities[0]] * 0.8 + pl.loc[:,priorities[1]] * 0.6 + pl.loc[:,priorities[2]] * 0.4 + pl[priorities[3]] * 0.2
    data_orig["index"] = indexes
    data_orig = data_orig.sort_values("index",ascending=False)
    suggestion = data_orig.iloc[:5,:-1]

    return suggestion

_A continuación se muestra un ejemplo del modelo el uso, donde se asume que un cliente llena la siguiente información en el formulario:_

- __Preguntas técnicas:__ Apartamento, En construcción, 100 metros, 3 habitaciones, 2 baños y 1 parqueadero.

- __Prioridades:__ Lo más importante para el cliente, de mayor a menor, es el estrato, el precio, los años de construido, el área y la existencia de zonas verdes.

_Una vez se corre el modelo, podemos ver las 5 sugerencias._

In [327]:
input_y=[[0,5,100,3,2,1]]
input_y = (input_y-np.array(X_mean))/np.array(X_std)
priorities=["estrato","precio","anos_construido","area","patio"]
suggestion = predict(input_y,priorities,data,data_orig)
suggestion

[17]


,tipo,anos_construido,area,habitaciones,banos,parqueaderos,estrato,patio,precio
4746,apartamento,en-construccion,160.0,2.0,2.0,2.0,4.0,No,"$1,320,000,000"
4326,apartamento,en-construccion,130.4,2.0,2.0,2.0,4.0,No,"$1,040,000,000"
3580,apartamento,en-construccion,106.0,2.0,2.0,2.0,4.0,No,"$750,000,000"
2347,apartamento,en-construccion,64.0,2.0,2.0,2.0,4.0,Sí,"$465,000,000"
3320,apartamento,en-construccion,97.0,2.0,2.0,2.0,4.0,No,"$680,000,000"


__Basado en la implementación anterior proponga como crear un feedback que alimente el sistema e impleméntelo.__

_Una vez se coloque el modelo en uso, se debe trabajar en la creación de una nueva base de datos que contenga, los datos demográficos del cliente, así como las características explícitas del inmueble seleccionado._

### Modelo #2: Incluye feedback

__Seleccione un nuevo método para analizar el problema y explique el porqué de su elección.__

_Después de conseguir una cantidad importante de observaciones (feedback), se puede pasar a entrenar un modelo de tipo __supervisado__, donde las elecciones de clientes antigüos, puedan ser utilizadas para la predicción de las elecciones de clientes futuros._

_Teniendo en cuenta que la decisión de adquirir un inmueble consiste en una sucesión de decisiones en cuanto a las diferentes variables que tiene cada inmueble, se estima que un algoritmo de __árbol de decisión__ puede entender los patrones detrás de cada decisión._

__Basado en el punto anterior proponga un modelo que permita solucionar el caso de estudio y explique cuál es la importancia de cada ítem del modelo, cuanto aporta porcentualmente cada ítem al modelo y cuál fue la razón de asignarle esos pesos a cada uno.__

_Teniendo en cuenta que son 6 las variables técnicas que se almacenan del inmueble seleccionado por cada cliente, se crean 6 árboles de decisión, cada uno de ellos teniendo en cuenta los datos demográficos de los clientes y una de las variables del inmueble como clase._

_Una vez se entrena el modelo, este funciona de la siguiente manera:_
- Se solicita la información demográfica y la clasificación de prioridades del cliente nuevo.
- Cada uno de los árboles de decisión (6 en total) determina el valor ideal de cada variable, de acuerdo con la información demográfica del cliente (ej: casa, de 5 a 10 años, 120 m, 3 habs, 2 baños y 1 parquedero).
- Conociendo los valores de cada variable explícita, se procede a filtrar los inmuebles que cumplen con dichas condiciones.
- Una vez filtrados, se seleccionan los inmuebles a sugerir basados en las prioridades de los clientes, utilizando la misma fórmula explicada en el ejercicio anterior.

__Implemente la solución en Python al problema propuesto indicando la versión de Python usada para el ejercicio.__

In [368]:
#Python 3.0
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor

In [394]:
#Read data
demo_data = pd.read_csv("metrocuadrado_feedback.csv")
property_database = pd.read_csv("metrocuadrado.csv")

X = demo_data.iloc[:,:6]
y_anos = demo_data.loc[:,"anos_construido"]
y_tipo = demo_data.loc[:,"tipo"]
y_area = demo_data.loc[:,"area"]
y_habs = demo_data.loc[:,"habitaciones"]
y_banos = demo_data.loc[:,"banos"]
y_parq = demo_data.loc[:,"parqueaderos"]

In [395]:
#Train model
est_anos = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
est_anos.fit(X, y_anos)
est_tipo = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
est_tipo.fit(X, y_tipo)
est_area = DecisionTreeRegressor(max_leaf_nodes=3, random_state=0)
est_area.fit(X, y_area)
est_habs = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
est_habs.fit(X, y_habs)
est_banos = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
est_banos.fit(X, y_banos)
est_parq = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
est_parq.fit(X, y_parq)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=3, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [399]:
def predict_fb(demo,priorities,demo_data,property_database):
    
    #Predict values
    anos = est_anos.predict(demo)
    tipo = est_tipo.predict(demo)
    area = est_area.predict(demo)
    habs = est_habs.predict(demo)
    banos = est_banos.predict(demo)
    parq = est_parq.predict(demo)

    #Filter out properties
    pl = demo_data[(demo_data.anos_construido==anos[0]) & (demo_data.tipo==tipo[0]) & (demo_data.habitaciones==habs[0]) & (demo_data.banos>=banos[0]) & (demo_data.parqueaderos>=parq[0]) & (demo_data.area > (area[0]-10)) & (demo_data.area < (area[0]+50))]
    pl["area"] = (pl["area"]-pl["area"].min())/pl["area"].max()
    pl["anos_construido"] = (pl["anos_construido"]-pl["anos_construido"].min())/pl["anos_construido"].max()
    pl["estrato"] = (pl["estrato"]-pl["estrato"].min())/pl["estrato"].max()
    pl["precio"] = ((1/pl.loc[:,"precio"])-(1/pl.loc[:,"precio"]).min())/(1/pl.loc[:,"precio"]).max()
    
    #Select 5 properties according to weights
    indexes = pl.loc[:,priorities[0]] * 0.8 + pl.loc[:,priorities[1]] * 0.6 + pl.loc[:,priorities[2]] * 0.4 + pl[priorities[3]] * 0.2
    property_database["index"] = indexes
    property_database = property_database.sort_values("index",ascending=False)
    suggestion = property_database.iloc[:5,:-1]

    return suggestion

_A continuación se muestra un ejemplo del modelo el uso, donde se evalúa un cliente con las siguiente características demográficas y prioridades:_

- __Información demográfica:__ Masculino (1),26 años (26), Casado (1), Posgrado (4), Sin hijos (0), Sin carro (0).

- __Prioridades:__ Lo más importante para el cliente, de mayor a menor, es el estrato, el precio, los años de construido, el área y la existencia de zonas verdes.

_Una vez se corre el modelo, podemos ver las 5 sugerencias._

In [400]:
demo = [[1,26,1,4,0,0]]
priorities = ["estrato","precio","anos_construido","area","patio"]
suggestion = predict_fb(demo,priorities,demo_data,property_database)
suggestion

,tipo,anos_construido,area,habitaciones,banos,parqueaderos,estrato,patio,precio
1124,apartamento,Más de 20 años,70.0,3.0,2.0,1.0,3.0,No,"$285,000,000"
1417,apartamento,Más de 20 años,82.0,3.0,2.0,1.0,3.0,Sí,"$330,000,000"
1416,apartamento,Más de 20 años,82.0,3.0,3.0,1.0,3.0,Sí,"$330,000,000"
1269,apartamento,Más de 20 años,72.0,3.0,2.0,2.0,3.0,No,"$310,000,000"
1268,apartamento,Más de 20 años,72.0,3.0,2.0,2.0,3.0,Sí,"$310,000,000"
